# Hugging Face Transformers 微调训练入门

本示例将介绍基于 Transformers 实现模型微调训练的主要流程，包括：
- 数据集下载
- 数据预处理
- 训练超参数配置
- 训练评估指标设置
- 训练器基本介绍
- 实战训练
- 模型保存

## YelpReviewFull 数据集

**Hugging Face 数据集：[ YelpReviewFull ](https://huggingface.co/datasets/yelp_review_full)**

### 数据集摘要

Yelp评论数据集包括来自Yelp的评论。它是从Yelp Dataset Challenge 2015数据中提取的。

### 支持的任务和排行榜
文本分类、情感分类：该数据集主要用于文本分类：给定文本，预测情感。

### 语言
这些评论主要以英语编写。

### 数据集结构

#### 数据实例
一个典型的数据点包括文本和相应的标签。

来自YelpReviewFull测试集的示例如下：

```json
{
    'label': 0,
    'text': 'I got \'new\' tires from them and within two weeks got a flat. I took my car to a local mechanic to see if i could get the hole patched, but they said the reason I had a flat was because the previous patch had blown - WAIT, WHAT? I just got the tire and never needed to have it patched? This was supposed to be a new tire. \\nI took the tire over to Flynn\'s and they told me that someone punctured my tire, then tried to patch it. So there are resentful tire slashers? I find that very unlikely. After arguing with the guy and telling him that his logic was far fetched he said he\'d give me a new tire \\"this time\\". \\nI will never go back to Flynn\'s b/c of the way this guy treated me and the simple fact that they gave me a used tire!'
}
```

#### 数据字段

- 'text': 评论文本使用双引号（"）转义，任何内部双引号都通过2个双引号（""）转义。换行符使用反斜杠后跟一个 "n" 字符转义，即 "\n"。
- 'label': 对应于评论的分数（介于1和5之间）。

#### 数据拆分

Yelp评论完整星级数据集是通过随机选取每个1到5星评论的130,000个训练样本和10,000个测试样本构建的。总共有650,000个训练样本和50,000个测试样本。

## 下载数据集

In [1]:
from datasets import load_dataset

dataset = load_dataset("yelp_review_full")

In [2]:
dataset

DatasetDict({
    train: Dataset({
        features: ['label', 'text'],
        num_rows: 650000
    })
    test: Dataset({
        features: ['label', 'text'],
        num_rows: 50000
    })
})

In [3]:
dataset["train"][10]

{'label': 0,
 'text': "Owning a driving range inside the city limits is like a license to print money.  I don't think I ask much out of a driving range.  Decent mats, clean balls and accessible hours.  Hell you need even less people now with the advent of the machine that doles out the balls.  This place has none of them.  It is april and there are no grass tees yet.  BTW they opened for the season this week although it has been golfing weather for a month.  The mats look like the carpet at my 107 year old aunt Irene's house.  Worn and thread bare.  Let's talk about the hours.  This place is equipped with lights yet they only sell buckets of balls until 730.  It is still light out.  Finally lets you have the pit to hit into.  When I arrived I wasn't sure if this was a driving range or an excavation site for a mastodon or a strip mining operation.  There is no grass on the range. Just mud.  Makes it a good tool to figure out how far you actually are hitting the ball.  Oh, they are cash 

In [4]:
import random
import pandas as pd
import datasets
from IPython.display import display, HTML

In [5]:
def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, datasets.ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))

In [8]:
show_random_elements(dataset["train"])

,label,text
0,1 star,"2 large cokes for almost $5 !?!?\n\nHell with you, czar of sirloin! I'm going to McDonalds, Carl's Jr or QuikTrip from now on."
1,3 stars,"OK buffet @40 bucks.\n\nExtremely crowded and very long lines make for a so so experience.\n\nI had a line pass,but if I didn't ,there would be no way I would stand in line for an hour to give them $40 bucks to eat an OK buffet.\n\nI've eaten at other places that are better and cheaper,but there is something about the folks bragging that they ate at the Bellagio that makes this some sort of Mecca.\n\nNot a must do."
2,4 stars,This was my first visit here. Liked the location which is quiet on a busy street. Probably quiet because of the time of year. People don't usually hang around for the 120 degree temperatures here. Loved the banana and the chocolate custard. Tried coffee with pieces of chocolate in it which I did not like as much because it wasnt as smooth and creamy as the banana and chocolate were. I will definitely be back for more. Forget about ice cream!
3,4 stars,"I love this Smashburger in the northwest valley, I love the concept, I love the modern & clean look, I love the sweet potato fries, and most of all I love the burgers. Very consistent, taste, flavorful burger. I have had the Arizona burger which isn't too bad here the other day, but my favorite is just the regular cheese burger. I have never had a bad burger here but I have only been to this location once since is barely opened last month. I have been to the 20th Street & Camelback location several times. I also like the fact that they are open at 10am, which is about the time I am hungry since my day starts way before most peeps. The service was good for me (sorry Bryan B, but I hope you give them another shot) I never wait more than a few minutes for my meal but I usually stop in when it is not peak lunch hours either."
4,4 stars,"After Earl and I had a bowl of hot savory ramen at Monta Chaya, we stopped by Frost Bite to get some shaved ice to balance out our taste buds.\n\nEarl got the medium with tiger blood and cucumber lime, topped with condensed milk. I must say the cucumber lime really tasted like cucumber lime. Even the smell was like the fruit/gourd itself! Highly recommend it with condensed milk. They not only topped it, but the added the condensed milk in the middle of the cup too. They do not hold back, and we LOVE it.\n\nI had the medium with coconut, tiger blood, and egg custard, with icecream, though, I was kind of disappointed because I was looking forward to the icecream at the bottom, but there was no icecream to be found. I was probably because initially, I wanted wedding cake flavor, but because I changed it to egg custard, the friendly cashier didn't take down my icecream order. When I told Earl about my no icecream, he said, \""We will go back soon enough for you to have it with icecream.\"" Yes-that's how much Earl enjoys it.\n\nNeedless to say, we are fans and will come back for more."
5,1 star,"Adorable Burros- Horrible Customer Service\n\nI waited quite awhile before I wrote this review as I was severely irritated right after the incident..\nI contacted this business to inquire about their services on initially on 3-4-14. \n\nOn 5-20 I had worked out the logistics/ got the appropriate permissions from our wedding venue and told Vanessa that I would like to book two Burros. \n\nOne month before our wedding date I received an e-mail from Vanessa letting me know that they had me as a tentative hold and that they needed the contract finalized. Over the next couple days we worked out the specifics as far as timing would go. \n\nOn October 1st I received an e-mail from Vanessa saying that they had overbooked and would only have one Burro available- to try and make it right they offered us their \""photo option\"". \nOn October 5th Vanessa sent me a pay pal invoice and it and the contract were returned on the same day. \n\nThe day of the event came (10/18) and even though the Bur

## 预处理数据

下载数据集到本地后，使用 Tokenizer 来处理文本，对于长度不等的输入数据，可以使用填充（padding）和截断（truncation）策略来处理。

Datasets 的 `map` 方法，支持一次性在整个数据集上应用预处理函数。

下面使用填充到最大长度的策略，处理整个数据集：

In [6]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")


def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)


tokenized_datasets = dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [7]:
show_random_elements(tokenized_datasets["train"], num_examples=1)

,label,text,input_ids,token_type_ids,attention_mask
0,4 stars,"This is a great affordable and centrally located hotel in the Montreal area. There is free wi-fi, and they give you the password on checkin, so there's no hassle in obtaining it. We needed a new iron since the one in the room didn't work. It arrived promptly in the morning. The staff is courtesy, and the rooms are clean and well appointed. Not a luxury hotel, but an awesome deal for your money. I would definitely stay here again.","[101, 1188, 1110, 170, 1632, 19017, 1105, 2129, 1193, 1388, 3415, 1107, 1103, 5206, 1298, 119, 1247, 1110, 1714, 192, 1182, 118, 20497, 117, 1105, 1152, 1660, 1128, 1103, 25241, 1113, 4031, 1394, 117, 1177, 1175, 112, 188, 1185, 1144, 27112, 1107, 11621, 1122, 119, 1284, 1834, 170, 1207, 3926, 1290, 1103, 1141, 1107, 1103, 1395, 1238, 112, 189, 1250, 119, 1135, 2474, 13796, 1107, 1103, 2106, 119, 1109, 2546, 1110, 15897, 117, 1105, 1103, 4045, 1132, 4044, 1105, 1218, 1923, 119, 1753, 170, 9886, 3415, 117, 1133, 1126, 14918, 2239, 1111, 1240, 1948, 119, 146, 1156, 5397, 2215, 1303, ...]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...]","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...]"


### 数据抽样

使用 1000 个数据样本，在 BERT 上演示小规模训练（基于 Pytorch Trainer）

`shuffle()`函数会随机重新排列列的值。如果您希望对用于洗牌数据集的算法有更多控制，可以在此函数中指定generator参数来使用不同的numpy.random.Generator。

In [8]:
small_train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(1000))
small_eval_dataset = tokenized_datasets["test"].shuffle(seed=42).select(range(1000))

half_train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(300000))
half_eval_dataset = tokenized_datasets["test"].shuffle(seed=42).select(range(30000))

## 微调训练配置

### 加载 BERT 模型

警告通知我们正在丢弃一些权重（`vocab_transform` 和 `vocab_layer_norm` 层），并随机初始化其他一些权重（`pre_classifier` 和 `classifier` 层）。在微调模型情况下是绝对正常的，因为我们正在删除用于预训练模型的掩码语言建模任务的头部，并用一个新的头部替换它，对于这个新头部，我们没有预训练的权重，所以库会警告我们在用它进行推理之前应该对这个模型进行微调，而这正是我们要做的事情。

In [9]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("bert-base-cased", num_labels=5)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initi

### 训练超参数（TrainingArguments）

完整配置参数与默认值：https://huggingface.co/docs/transformers/v4.36.1/en/main_classes/trainer#transformers.TrainingArguments

源代码定义：https://github.com/huggingface/transformers/blob/v4.36.1/src/transformers/training_args.py#L161

**最重要配置：模型权重保存路径(output_dir)**

In [10]:
from transformers import TrainingArguments

model_dir = "models/bert-base-cased"

# logging_steps 默认值为500，根据我们的训练数据和步长，将其设置为100
training_args = TrainingArguments(output_dir=f"{model_dir}/test_trainer",
                                  logging_dir=f"{model_dir}/test_trainer/runs",
                                  logging_steps=100)

2024-01-08 01:53:21.100205: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-08 01:53:21.100261: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-08 01:53:21.101482: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-08 01:53:21.108728: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-01-08 01:53:22.106631: W tensorflow/compiler/tf2

In [12]:
# 完整的超参数配置
print(training_args)

TrainingArguments(
_n_gpu=4,
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
auto_find_batch_size=False,
bf16=False,
bf16_full_eval=False,
data_seed=None,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_pin_memory=True,
ddp_backend=None,
ddp_bucket_cap_mb=None,
ddp_find_unused_parameters=None,
ddp_timeout=1800,
debug=[],
deepspeed=None,
disable_tqdm=False,
do_eval=False,
do_predict=False,
do_train=False,
eval_accumulation_steps=None,
eval_delay=0,
eval_steps=None,
evaluation_strategy=IntervalStrategy.NO,
fp16=False,
fp16_backend=auto,
fp16_full_eval=False,
fp16_opt_level=O1,
fsdp=[],
fsdp_config={'fsdp_min_num_params': 0, 'xla': False, 'xla_fsdp_grad_ckpt': False},
fsdp_min_num_params=0,
fsdp_transformer_layer_cls_to_wrap=None,
full_determinism=False,
gradient_accumulation_steps=1,
gradient_checkpointing=False,
greater_is_better=None,
group_by_length=False,
half_precision_backend=auto,
hub_model_id=None,
hub_private_repo=False,
hub_strategy=HubSt

### 训练过程中的指标评估（Evaluate)

**[Hugging Face Evaluate 库](https://huggingface.co/docs/evaluate/index)** 支持使用一行代码，获得数十种不同领域（自然语言处理、计算机视觉、强化学习等）的评估方法。 当前支持 **完整评估指标：https://huggingface.co/evaluate-metric**

训练器（Trainer）在训练过程中不会自动评估模型性能。因此，我们需要向训练器传递一个函数来计算和报告指标。 

Evaluate库提供了一个简单的准确率函数，您可以使用`evaluate.load`函数加载

In [11]:
import numpy as np
import evaluate

metric = evaluate.load("accuracy")


接着，调用 `compute` 函数来计算预测的准确率。

在将预测传递给 compute 函数之前，我们需要将 logits 转换为预测值（**所有Transformers 模型都返回 logits**）。

In [12]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

#### 训练过程指标监控

通常，为了监控训练过程中的评估指标变化，我们可以在`TrainingArguments`指定`evaluation_strategy`参数，以便在 epoch 结束时报告评估指标。

In [13]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(output_dir=f"{model_dir}/test_trainer",
                                  logging_dir=f"{model_dir}/test_trainer/runs",
                                  evaluation_strategy="steps",  # 可选值：no, steps, epoch
                                  eval_steps=500,
                                  per_device_eval_batch_size=24,
                                  per_device_train_batch_size=24,
                                  logging_steps=500)

## 开始训练

### 实例化训练器（Trainer）

`kernel version` 版本问题：暂不影响本示例代码运行

In [14]:
trainer = Trainer(
    model=model,
    args=training_args,
    # train_dataset=small_train_dataset,
    # eval_dataset=small_eval_dataset,
    train_dataset=half_train_dataset,
    eval_dataset=half_eval_dataset,
    compute_metrics=compute_metrics,
)

Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


## 使用 nvidia-smi 查看 GPU 使用

为了实时查看GPU使用情况，可以使用 `watch` 指令实现轮询：`watch -n 1 nvidia-smi`:

```shell
Every 1.0s: nvidia-smi                                                   Wed Dec 20 14:37:41 2023

Wed Dec 20 14:37:41 2023
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.129.03             Driver Version: 535.129.03   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:0D.0 Off |                    0 |
| N/A   64C    P0              69W /  70W |   6665MiB / 15360MiB |     98%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+----------------------+

+---------------------------------------------------------------------------------------+
| Processes:                                                                            |
|  GPU   GI   CI        PID   Type   Process name                            GPU Memory |
|        ID   ID                                                             Usage      |
|=======================================================================================|
|    0   N/A  N/A     18395      C   /root/miniconda3/bin/python                6660MiB |
+---------------------------------------------------------------------------------------+
```

In [17]:
trainer.train()

/home/yuxiang/miniconda3/envs/py311_llm_dev/lib/python3.11/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
/home/yuxiang/miniconda3/envs/py311_llm_dev/lib/python3.11/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Step,Training Loss,Validation Loss,Accuracy
500,0.962200,0.815489,0.642333
1000,0.824400,0.794675,0.651033
1500,0.796200,0.756103,0.667400
2000,0.771300,0.756934,0.664933
2500,0.756800,0.751870,0.668400
3000,0.746500,0.739780,0.671567
3500,0.683100,0.729674,0.680867
4000,0.663900,0.738611,0.679967
4500,0.655300,0.734863,0.678200
5000,0.653900,0.730346,0.679167


/home/yuxiang/miniconda3/envs/py311_llm_dev/lib/python3.11/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/home/yuxiang/miniconda3/envs/py311_llm_dev/lib/python3.11/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/home/yuxiang/miniconda3/envs/py311_llm_dev/lib/python3.11/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/home/yuxiang/miniconda3/envs/py311_llm_dev/lib/python3.11/site-packages/torch/

TrainOutput(global_step=9375, training_loss=0.6641348388671875, metrics={'train_runtime': 12425.2809, 'train_samples_per_second': 72.433, 'train_steps_per_second': 0.755, 'total_flos': 2.368063282176e+17, 'train_loss': 0.6641348388671875, 'epoch': 3.0})

In [18]:
small_test_dataset = tokenized_datasets["test"].shuffle(seed=64).select(range(100))

half_test_dataset = tokenized_datasets["test"].shuffle(seed=64).select(range(30000))

In [19]:
# trainer.evaluate(small_test_dataset)
trainer.evaluate(half_test_dataset)

{'eval_loss': 0.7699790596961975,
 'eval_accuracy': 0.6858666666666666,
 'eval_runtime': 133.0383,
 'eval_samples_per_second': 225.499,
 'eval_steps_per_second': 2.353,
 'epoch': 3.0}

### 保存模型和训练状态

- 使用 `trainer.save_model` 方法保存模型，后续可以通过 from_pretrained() 方法重新加载
- 使用 `trainer.save_state` 方法保存训练状态

In [20]:
trainer.save_model(f"{model_dir}/finetuned-trainer")

In [21]:
trainer.save_state()

## Homework: 使用完整的 YelpReviewFull 数据集训练，对比看 Acc 最高能到多少

In [15]:
# Tokenize 全量数据集

full_train_dataset = tokenized_datasets["train"].shuffle(seed=42)
full_eval_dataset = tokenized_datasets["test"].shuffle(seed=42)

In [16]:
# 设置训练超参数

from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(output_dir=f"{model_dir}/homework_test_trainer",
                                  logging_dir=f"{model_dir}/homework_test_trainer/runs",
                                  evaluation_strategy="steps",  # 可选值：no, steps, epoch
                                  eval_steps=3000,
                                  per_device_eval_batch_size=24,
                                  per_device_train_batch_size=24,
                                  logging_steps=3000)

In [17]:
# 实例化训练器（Trainer）

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=full_train_dataset,
    eval_dataset=full_eval_dataset,
    compute_metrics=compute_metrics,
)

Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


In [18]:
# 开始训练

trainer.train()

/home/yuxiang/miniconda3/envs/py311_llm_dev/lib/python3.11/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
/home/yuxiang/miniconda3/envs/py311_llm_dev/lib/python3.11/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Step,Training Loss,Validation Loss,Accuracy
3000,0.855800,0.773281,0.657140
6000,0.771700,0.759693,0.669780
9000,0.752700,0.728147,0.680160
12000,0.734800,0.716574,0.684020
15000,0.695200,0.727479,0.683000
18000,0.660400,0.718358,0.686660
21000,0.655400,0.713631,0.690300
24000,0.653400,0.705623,0.691960
27000,0.645200,0.694544,0.695500
30000,0.562600,0.738102,0.692760


/home/yuxiang/miniconda3/envs/py311_llm_dev/lib/python3.11/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/home/yuxiang/miniconda3/envs/py311_llm_dev/lib/python3.11/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/home/yuxiang/miniconda3/envs/py311_llm_dev/lib/python3.11/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/home/yuxiang/miniconda3/envs/py311_llm_dev/lib/python3.11/site-packages/torch/

TrainOutput(global_step=40626, training_loss=0.6581256824952193, metrics={'train_runtime': 45000.0574, 'train_samples_per_second': 43.333, 'train_steps_per_second': 0.903, 'total_flos': 5.130803778048e+17, 'train_loss': 0.6581256824952193, 'epoch': 3.0})

In [19]:
# 评估模型最终准确率

full_test_dataset = tokenized_datasets["test"].shuffle(seed=64)
trainer.evaluate(full_test_dataset)

{'eval_loss': 0.7362170219421387,
 'eval_accuracy': 0.69534,
 'eval_runtime': 397.1876,
 'eval_samples_per_second': 125.885,
 'eval_steps_per_second': 2.623,
 'epoch': 3.0}

In [20]:
# 保存模型和训练状态

trainer.save_model(f"{model_dir}/homework-finetuned-trainer")
trainer.save_state()